In [ ]:
!wget https://s3.amazonaws.com/my89public/quac/train_v0.2.json

--2022-05-26 06:43:56--  https://s3.amazonaws.com/my89public/quac/train_v0.2.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.1.238
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.1.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68114819 (65M) [application/json]
Saving to: ‘train_v0.2.json’

train_v0.2.json     100%[===================>]  64.96M  13.3MB/s    in 6.3s    

2022-05-26 06:44:04 (10.3 MB/s) - ‘train_v0.2.json’ saved [68114819/68114819]



In [ ]:
!wget https://s3.amazonaws.com/my89public/quac/val_v0.2.json

--2022-05-26 06:44:04--  https://s3.amazonaws.com/my89public/quac/val_v0.2.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.11.142
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.11.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8929167 (8.5M) [application/json]
Saving to: ‘val_v0.2.json’

val_v0.2.json       100%[===================>]   8.51M  4.39MB/s    in 1.9s    

2022-05-26 06:44:07 (4.39 MB/s) - ‘val_v0.2.json’ saved [8929167/8929167]



# Dependencies

In [ ]:
import json

In [ ]:
with open('/content/train_v0.2.json', 'r') as f:
    train=json.load(f)

with open('/content/val_v0.2.json', 'r') as f:
    dev=json.load(f)

# train[:2], dev[:2]
train['data'][:2]

[{'background': 'The Malayali people or Keralite people (also spelt Malayalee, Malayalam script: mlyaalli and keerlliiy[?]) are an Indian ethnic group originating from the present-day state of Kerala, located in South India. They are identified as native speakers of the Malayalam language, which is classified as part of the Dravidian family of languages. As they primarily live in Kerala, the word Keralite is used as an alternative to Malayali. According to the Indian census of 2011, there are approximately 33 million Malayalis in Kerala, making up 96.7% of the total population of the state.',
  'paragraphs': [{'context': 'According to the Indian census of 2001, there were 30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India, and 96.7% of the total population of the state. There were a further 701,673 (2.1% of the total number) in Karnataka, 557,705 (1.7%) in Tamil Nadu and 406,358 (1.2%) in Maharashtra. The number of Malayalam s

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 21.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-bx85r5ry/farm-haystack_d827dd6debbd4ab589e6750cb35bcca9
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-bx85r5ry/farm-haystack_d827dd6debbd4ab589e6750cb35bcca9
  Resolved https://github.com/deepset-ai/haystack.git to commit b2a2c10fae6944e8b38b7e72ef68a3f5f2586d6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 37

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore
doc_store=ElasticsearchDocumentStore(
    host='localhost', username='', password='', index='quac_docs'
)

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [ ]:
import requests

res=requests.get('http://localhost:9200/_cluster/health')
res.json()

{'active_primary_shards': 2,
 'active_shards': 2,
 'active_shards_percent_as_number': 50.0,
 'cluster_name': 'elasticsearch',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'yellow',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 2}

In [ ]:
train['data'][0]['paragraphs'][0]

In [ ]:
docs=[]

for i in train['data']:
    for j in i['paragraphs']:
        context=j['context']
        questions=j['qas']
        for ques in questions:
            answer=ques['answers'][0]['text']
            question=ques['question']

            docs.append({
                'context':context,
                'questions':question,
                'answer':answer
            })

In [ ]:
quac_docs=[]

for sample in docs[:50000]:
    c=sample['context']
    
    quac_docs.append({
        'content':c
    })


In [ ]:
doc_store.write_documents(quac_docs)

In [ ]:
res=requests.get('http://localhost:9200/_cluster/health')
res

<Response [200]>

In [ ]:
res=requests.get('http://localhost:9200/_count')
res.json()

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 2, 'total': 2},
 'count': 4914}

In [ ]:
requests.post('http://localhost:9200/quac_docs/_delete_by_query',
              json={
                  'query':{
                      'match_all':{}
                  }
              })

<Response [200]>

In [ ]:
context=[sample['context'] for sample in docs]
context=list(set(context))
len(context)

6843

In [ ]:
quac_docs=[]
for sample in context:
    quac_docs.append({'content':sample})
len(quac_docs)

6843

In [ ]:
doc_store.write_documents(quac_docs)

In [ ]:
requests.get('http://localhost:9200/_count').json()

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 2, 'total': 2},
 'count': 6843}

In [ ]:
from haystack.retriever.dense import DensePassageRetriever


retriever=DensePassageRetriever(
    document_store=doc_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-ctx_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-ctx_encoder-single-nq-base


In [ ]:
reader=FARMReader(
    model_name_or_path='deepset/bert-base-cased-squad2',
    use_gpu=True
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/bert-base-cased-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/bert-base-cased-squad2


Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [ ]:
doc_store.update_embeddings(retriever=retriever)

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 6843 docs ...


Updating embeddings:   0%|          | 0/6843 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/6848 [00:00<?, ? Docs/s]

In [ ]:
from haystack.pipeline import ExtractiveQAPipeline
pipeline=ExtractiveQAPipeline(reader=reader, retriever=retriever)
pipeline.run(query='How did they target her email?', params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 16.05 Batches/s]


{'answers': [<Answer {'answer': 'Twitter videos and offering her support for fellow victims of sexual harassment', 'type': 'extractive', 'score': 0.51261106133461, 'context': " fans, thanking them in a series of Twitter videos and offering her support for fellow victims of sexual harassment. She also criticized Fox's attempt", 'offsets_in_document': [{'start': 926, 'end': 1005}], 'offsets_in_context': [{'start': 36, 'end': 115}], 'document_id': '494341e634939af60e84fcaad5bbc9c5', 'meta': {}}>,
  <Answer {'answer': 'anthrax hoax', 'type': 'extractive', 'score': 0.5093208849430084, 'context': 'On October 12, 2001, Miller opened an anthrax hoax letter mailed to her New York Times office. The 2001 anthrax attacks had begun occurring in the wak', 'offsets_in_document': [{'start': 38, 'end': 50}], 'offsets_in_context': [{'start': 38, 'end': 50}], 'document_id': 'fa5b7b22a77d584267305b8cadbcff03', 'meta': {}}>,
  <Answer {'answer': 'government shutdown', 'type': 'extractive', 'score': 0.49399